In [ ]:
# Product Recommendation System for e-commerce
import argparse
import time
import sys
import warnings
import logging

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
from joblib import dump, load
import pandas as pd

In [ ]:
#Performance metrics
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install tensorflow_text
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import tensorflow_hub as hub
!pip install tensorflow-gpu
import tensorflow as tf
from tensorflow import keras
import tensorflow_text
import sklearn.metrics as metrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 52.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Fo

In [ ]:
!pip install nltk
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import json
import ast
#to filter the specifications column

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.8 MB/s eta 0:00:00


In [ ]:
from transformers import DistilBertModel, DistilBertTokenizer
import torch

import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

In [ ]:
DATASET_FILE = "/content/drive/MyDrive/flipkart_com-ecommerce_sample.csv"
TRUE_K =12

In [ ]:
no_rows = 20000
df = pd.read_csv(DATASET_FILE,nrows=no_rows)
df = df.drop_duplicates(subset = ["pid"])
df = df[['product_name','description','brand','discounted_price','product_specifications']]
df2 = df['product_name']
df = df.astype(str)
df = df.dropna()
df = df.applymap(lambda x: x.lower())
# print(df['description'][:5])

In [ ]:
df['product_specifications'][0]

'{"product_specification"=>[{"key"=>"number of contents in sales package", "value"=>"pack of 3"}, {"key"=>"fabric", "value"=>"cotton lycra"}, {"key"=>"type", "value"=>"cycling shorts"}, {"key"=>"pattern", "value"=>"solid"}, {"key"=>"ideal for", "value"=>"women\'s"}, {"value"=>"gentle machine wash in lukewarm water, do not bleach"}, {"key"=>"style code", "value"=>"altht_3p_21"}, {"value"=>"3 shorts"}]}'

In [ ]:
import re
# Function to extract keys and values from product_specifications column
def extract_specifications(specifications):
    pairs = re.findall(r'"key"=>"(.*?)", "value"=>"(.*?)"', specifications)
    pairs_formatted = [f"{key}: {value}" for key, value in pairs]
    return ' '.join(pairs_formatted)

# Apply the function to the product_specifications column
df['product_specifications'] = df['product_specifications'].apply(extract_specifications)

print(df['product_specifications'][0])

# df["product_specifications"][0]

import re

def remove_colors(brand):
    # Define a list of colors to remove
    colors = [
    'amber', 'aquamarine', 'aqua', 'beige', 'black', 'blue', 'bronze',
    'brown', 'burgundy', 'caramel', 'chartreuse', 'charcoal', 'coral',
    'crimson', 'cyan', 'eggplant', 'emerald', 'forest green', 'fuchsia',
    'gold', 'golden', 'gray', 'green', 'indigo', 'ivory', 'jade', 'khaki',
    'lavender', 'lemon', 'lime', 'lilac', 'magenta', 'maroon', 'mint',
    'mint green', 'moss green', 'mustard', 'navy', 'neon amber',
    'neon blue', 'neon coral', 'neon crimson', 'neon fuchsia', 'neon gold',
    'neon green', 'neon indigo', 'neon lime', 'neon magenta', 'neon orange',
    'neon pink', 'neon platinum', 'neon purple', 'neon red', 'neon silver',
    'neon teal', 'neon turquoise', 'neon violet', 'olive', 'olive green',
    'orange', 'orchid', 'pearl', 'peach', 'peacock blue', 'peacock green',
    'periwinkle', 'pink', 'platinum', 'purple', 'red', 'rose', 'rose gold',
    'ruby', 'ruby red', 'rust', 'salmon', 'salmon pink', 'sapphire',
    'silver', 'sky blue', 'steel blue', 'tan', 'tangerine', 'taupe', 'teal',
    'topaz', 'turquoise', 'violet', 'white', 'yellow', 'baby pink', 'baby blue'
    ]
    # Create a regular expression pattern to match the colors
    pattern = r'\b(?:{})\b'.format('|'.join(colors))

    # Remove the colors from the brand name using regex substitution
    brand = re.sub(pattern, '', brand, flags=re.IGNORECASE).strip()

    return brand

# Apply the remove_colors function to the 'brand' column
df['brand'] = df['brand'].apply(remove_colors)







number of contents in sales package: pack of 3 fabric: cotton lycra type: cycling shorts pattern: solid ideal for: women's style code: altht_3p_21


In [ ]:
# print(df['product_category_tree'][0])

In [ ]:
# df['product_category_tree'] = df['product_category_tree'].str.replace("[", "").str.replace("]", "").str.replace("\"", "")

# print(df['product_category_tree'][0])

In [ ]:
df = df.applymap(lambda x: re.sub('[^A-Za-z0-9]+', ' ', str(x)))


# df['combined_text'] = df['product_name'] + ' ' + df['description'] + ' ' + df['brand'] #1
df['combined_text'] = df['product_name'] + ' ' + df['description'] + ' ' + df['product_specifications'] #2
# df['combined_text'] = df['description'] + ' ' + df['brand'] + ' ' + df['product_specifications']  #3
# df['combined_text'] = df['product_name'] + ' ' + df['description'] + ' ' + df['brand'] + ' ' + df['product_specifications'] #4
# df['combined_text'] = df['product_name'] #5


# df['combined_text'] = df['combined_text'].str.lower()
# df['combined_text'] = df['combined_text'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', str(x)))


In [ ]:
df['combined_text'][0]

'alisha solid women s cycling shorts key features of alisha solid women s cycling shorts cotton lycra navy red navy specifications of alisha solid women s cycling shorts shorts details number of contents in sales package pack of 3 fabric cotton lycra type cycling shorts general details pattern solid ideal for women s fabric care gentle machine wash in lukewarm water do not bleach additional details style code altht 3p 21 in the box 3 shorts number of contents in sales package pack of 3 fabric cotton lycra type cycling shorts pattern solid ideal for women s style code altht 3p 21'

In [ ]:
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('bert-base-nli-mean-tokens')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=68fb86807fd1485f8e12420d32e1c2415154003063036a4fda48aa4c87402822
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
# Get embeddings
sentence_embeddings = bert.encode(df['combined_text'].tolist())

In [ ]:
import numpy as np
# Define the file path where you want to save the embeddings
embeddings_path = "/content/drive/MyDrive/Colab Notebooks/embeddings2.npy"

# np.save(embeddings_path,sentence_embeddings)
# Save the embeddings to the file

In [ ]:
def extract_brand(query):
    query = str(query).lower()  # Convert the query to lowercase for case-insensitive matching
    brand_names = df['brand'].str.lower()  # Convert the 'brand' column to lowercase for case-insensitive matching

    for brand in brand_names:
        if brand in query:
            return brand

    return None  # If no brand name is found in the query
import numpy as np
embeddings_path = "/content/drive/MyDrive/Colab Notebooks/embeddings2.npy"
np.save(embeddings_path,sentence_embeddings)

# userinput = input("enter query") #test
# print(extract_brand(userinput))
# print(len(df['brand']))
# print(len(df))

In [ ]:
sentence_embeddings = np.load(embeddings_path) #for future sessions
#Compute similarity
from sklearn.metrics.pairwise import cosine_similarity

# user_input = str.lower(input("Enter your query: "))

# user_input = user_input.apply(lambda x: re.sub('[^A-Za-z0-9]+',' ',str(x))) since .apply() is for pd series / dataframes

import re
# start = time.time() #start time for recommendation
pd.set_option('display.max_colwidth', -1) #to prevent truncation from end of product name

def brand_inference(user_input,sentence_embeddings):
  print("Recommendations for \"",user_input,"\" are: ",sep='')
  user_input = re.sub('[^A-Za-z0-9]+', ' ', user_input)
  start=time.time()
  query_brand = extract_brand(user_input)
  user_input_embedding = bert.encode([user_input])
  # similarity=cosine_similarity(sentence_embeddings)
  similarities = cosine_similarity(user_input_embedding, sentence_embeddings)
  if query_brand is not None:
    df['brand_match_weight'] = df['brand'].apply(lambda x: 1.5 if x == query_brand else 1.0)
    df['brand_match_weight'] = df['brand_match_weight'] * df['product_name'].apply(lambda x: 1.5  if query_brand in x else 1.0) #now checking for brand name in product_name column too
  else:
    df['brand_match_weight'] = 1.0

  similarities_with_brand = similarities*(np.array(df['brand_match_weight']))
  num_recommendations = 10
  top_indices = similarities_with_brand.argsort()[0][::-1][:num_recommendations]

  df['product_name_origin']=df2 #to retain the original product names, since the df was preprocessed, which involved removing all symbols etc.
  recommended_products = df.iloc[top_indices]['product_name_origin']
  print(recommended_products)
  end=time.time()-start
  print(end)

# input_list = ["bluetooth headphones wired and wireless","asics running shoes","mast & harbour full sleeve ","mast & harbour womens","allure auto car mat"]
input_list = ["bluetooth headphones wired and wireless","asics running shoes","Car mat","T-Shirts", "Bluestone solitaire diamond 18k ring necklace jewellery"]

for i in input_list:
  brand_inference(i,sentence_embeddings)
  print()

# print(type(similarities))
# if query_brand!=None:
#   df['brand_match_weight'] = df['brand'].apply(lambda x: 10 if x == query_brand else 1.0)  #previously used just to match with brand column

# if query_brand is not None:
#     df['brand_match_weight'] = df['brand'].apply(lambda x: 1.5 if x == query_brand else 1.0)
#     df['brand_match_weight'] = df['brand_match_weight'] * df['product_name'].apply(lambda x: 1.5  if query_brand in x else 1.0) #now checking for brand name in product_name column too
# else:
#     df['brand_match_weight'] = 1.0

# print(df['brand_match_weight'][:10])
# similarities_df = pd.DataFrame(similarities,columns = df.index)
# similarities_np_array = np.array(similarities)
# similarities_with_brand = similarities*(np.array(df['brand_match_weight']))
# print(np.array(df['brand_match_weight'][:10]))
# print(similarities[:10])
# print(similarities_with_brand[:10])
# print(len(df['brand_match_weight']))
# num_recommendations = 10
# top_indices = similarities_with_brand.argsort()[0][::-1][:num_recommendations]
# top_indices = similarities.argsort()[0][::-1][:num_recommendations]

# top_indices = np.argsort(similarities_with_brand[0])[::-1][:num_recommendations]
# df['product_name_origin']=df2 #to retain the original product names, since the df was preprocessed, which involved removing all symbols etc.
# recommended_products = df.iloc[top_indices]['product_name_origin']

# print(recommended_products)
# end = time.time() - start #end time
# print(end)
# print(df.iloc[top_indices]['product_specifications'])

Recommendations for "bluetooth headphones wired and wireless" are: 
5507     Tech Yug BH-503 bluethooth headset Wireless Bluetooth Headset                                                      
19081    Head Kik Premium Quality Solo2 S460 Wireless Bluetooth Headset Stereo Dynamic Wired & Wireless Bluetooth Headphones
9163     Zidane 503TF_BLK Wireless Bluetooth Headset                                                                        
7955     LIFE LIKE STN-840 4.1 WITH MIC Wireless Bluetooth Headset                                                          
8007     LIFE LIKE OTP-200 WITH MIC Wireless Bluetooth Headset                                                              
7930     LIFE LIKE S450 3.0 WITH MIC GOOD SOUND QUALITY Wired & Wireless Bluetooth Headphones                               
9185     CHKOKKO Earbud Skin Wireless Bluetooth Headset                                                                     
7588     Janhvi Bluetooth Beatz Pill F Wireless Home Audi

In [ ]:
["bluetooth headphones wired and wireless","asics running shoes","mast & harbour full sleeve ","mast & harbour womens","allure auto car mat"]

['bluetooth headphones wired and wireless',
 'asics running shoes',
 'mast & harbour full sleeve ',
 'mast & harbour womens',
 'allure auto car mat']